In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
from sklearn.preprocessing import OneHotEncoder
import pickle
import xgboost as xgb

pd.set_option('display.max_columns', None)


In [2]:
df2days = pd.read_csv('/Users/afnanalamri/Renewable_Energy/raw_data/onemoth_sep_2days.csv')

In [3]:

df9 = pd.read_csv('/Users/afnanalamri/Renewable_Energy/raw_data/onemonth_sep.csv')

df9.head()


,time,temperature_2m (°C),relative_humidity_2m (%),wind_speed_10m (km/h),wind_speed_100m (km/h),soil_temperature_0_to_7cm (°C),soil_temperature_7_to_28cm (°C),soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C)
0,2024-09-01T00:00,30.4,30,30.6,46.8,31.6,37.8,36.2,32.5
1,2024-09-01T01:00,30.8,31,31.7,48.0,32.4,37.8,36.2,32.5
2,2024-09-01T02:00,30.1,34,32.1,49.9,31.6,37.7,36.2,32.5
3,2024-09-01T03:00,29.5,39,31.7,49.4,30.9,37.6,36.2,32.5
4,2024-09-01T04:00,29.5,41,31.8,48.7,30.5,37.4,36.2,32.5


In [4]:
import pandas as pd

# Concatenate DataFrame B on top of A
df = pd.concat([df2days, df9], ignore_index=True)

# Display the result
df

,time,temperature_2m (°C),relative_humidity_2m (%),wind_speed_10m (km/h),wind_speed_100m (km/h),soil_temperature_0_to_7cm (°C),soil_temperature_7_to_28cm (°C),soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C),diffuse_radiation (W/m²)
0,2024-08-30T00:00,31.7,40,29.5,45.4,33.6,38.8,36.2,32.4,0.0
1,2024-08-30T01:00,31.5,39,29.4,45.9,34.3,38.7,36.2,32.4,0.0
2,2024-08-30T02:00,31.1,38,29.4,46.5,33.3,38.6,36.2,32.4,0.0
3,2024-08-30T03:00,30.6,36,30.2,47.6,32.5,38.5,36.2,32.4,0.0
4,2024-08-30T04:00,30.6,36,31.2,48.6,31.9,38.4,36.2,32.4,15.0
...,...,...,...,...,...,...,...,...,...,...
763,2024-09-30T19:00,30.1,34,27.2,41.8,32.9,35.3,35.0,33.4,NaN
764,2024-09-30T20:00,29.2,38,25.6,41.1,31.5,35.2,35.0,33.4,NaN
765,2024-09-30T21:00,28.5,42,28.0,44.3,30.4,35.2,35.0,33.4,NaN
766,2024-09-30T22:00,27.8,45,29.9,46.6,29.4,35.1,35.0,33.4,NaN


In [5]:
df.columns

Index(['time', 'temperature_2m (°C)', 'relative_humidity_2m (%)',
       'wind_speed_10m (km/h)', 'wind_speed_100m (km/h)',
       'soil_temperature_0_to_7cm (°C)', 'soil_temperature_7_to_28cm (°C)',
       'soil_temperature_28_to_100cm (°C)',
       'soil_temperature_100_to_255cm (°C)', 'diffuse_radiation (W/m²)'],
      dtype='object')

In [6]:
# df.rename(columns={
#     'temperature_2m (°C)': 'Temperature',
#     'relative_humidity_2m (%)': 'Humidity',
#     'wind_speed_10m (km/h)': 'WindSpeed',
# }, inplace=True)
# df = df[['Temperature', 'Humidity', 'WindSpeed']]
# df['WindSpeed']= df['WindSpeed']/ 3.6

In [7]:
# Create a DataFrame with a specific date (2024-10-20)
df['Date'] = pd.to_datetime(df['time'])

df.set_index('Date', inplace=True)

df.index = pd.to_datetime(df.index)
df['Date'] = df.index.date

KSA_holidays = holidays.SA(years=[2024])
df['IsHoliday'] = df['Date'].apply(lambda x: 1 if x in KSA_holidays else 0)

df= df.drop(columns='time')
df.head()

,temperature_2m (°C),relative_humidity_2m (%),wind_speed_10m (km/h),wind_speed_100m (km/h),soil_temperature_0_to_7cm (°C),soil_temperature_7_to_28cm (°C),soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C),diffuse_radiation (W/m²),Date,IsHoliday
Date,,,,,,,,,,,
2024-08-30 00:00:00,31.7,40,29.5,45.4,33.6,38.8,36.2,32.4,0.0,2024-08-30,0
2024-08-30 01:00:00,31.5,39,29.4,45.9,34.3,38.7,36.2,32.4,0.0,2024-08-30,0
2024-08-30 02:00:00,31.1,38,29.4,46.5,33.3,38.6,36.2,32.4,0.0,2024-08-30,0
2024-08-30 03:00:00,30.6,36,30.2,47.6,32.5,38.5,36.2,32.4,0.0,2024-08-30,0
2024-08-30 04:00:00,30.6,36,31.2,48.6,31.9,38.4,36.2,32.4,15.0,2024-08-30,0


In [8]:
df.rename(columns={
    'temperature_2m (°C)': 'Temperature',
    'relative_humidity_2m (%)': 'Humidity',
    'wind_speed_10m (km/h)': 'WindSpeed',
}, inplace=True)
df = df[['Temperature', 'Humidity', 'WindSpeed','IsHoliday' ]]
df['WindSpeed']= df['WindSpeed']/ 3.6
df

,Temperature,Humidity,WindSpeed,IsHoliday
Date,,,,
2024-08-30 00:00:00,31.7,40,8.194444,0
2024-08-30 01:00:00,31.5,39,8.166667,0
2024-08-30 02:00:00,31.1,38,8.166667,0
2024-08-30 03:00:00,30.6,36,8.388889,0
2024-08-30 04:00:00,30.6,36,8.666667,0
...,...,...,...,...
2024-09-30 19:00:00,30.1,34,7.555556,0
2024-09-30 20:00:00,29.2,38,7.111111,0
2024-09-30 21:00:00,28.5,42,7.777778,0


In [9]:
# Initialize the SA holidays for 2024
KSA_holidays = holidays.SA(years=[2024])

# Print all holidays in SA for 2024
for date, name in sorted(KSA_holidays.items()):
    print(date, name)


2024-02-22 يوم التأسيسي
2024-04-10 (تقدير) عطلة عيد الفطر
2024-04-11 (تقدير) عطلة عيد الفطر
2024-04-12 (تقدير) عطلة عيد الفطر
2024-04-13 (تقدير) عطلة عيد الفطر
2024-04-14 (تقدير ملاحظة) عطلة عيد الفطر
2024-04-15 (تقدير ملاحظة) عطلة عيد الفطر
2024-06-15 (تقدير) يوم عرفة
2024-06-16 (تقدير) عطلة عيد الأضحى
2024-06-17 (تقدير) عطلة عيد الأضحى
2024-06-18 (تقدير) عطلة عيد الأضحى
2024-06-19 (تقدير ملاحظة) عطلة عيد الأضحى
2024-09-23 اليوم الوطني


In [10]:
# Create new columns using the Datetime index
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Day'] = df.index.day
df['Hour'] = df.index.hour
# df = df[['IsHoliday']]


In [11]:
# Define the seasons
def categorize_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

df['Season'] = df['Month'].apply(categorize_season)

In [12]:
df.head()

,Temperature,Humidity,WindSpeed,IsHoliday,Year,Month,Day,Hour,Season
Date,,,,,,,,,
2024-08-30 00:00:00,31.7,40,8.194444,0,2024,8,30,0,Summer
2024-08-30 01:00:00,31.5,39,8.166667,0,2024,8,30,1,Summer
2024-08-30 02:00:00,31.1,38,8.166667,0,2024,8,30,2,Summer
2024-08-30 03:00:00,30.6,36,8.388889,0,2024,8,30,3,Summer
2024-08-30 04:00:00,30.6,36,8.666667,0,2024,8,30,4,Summer


In [13]:
# Define time of day categories
def categorize_time_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Night'

df['TimeOfDay'] = df['Hour'].apply(categorize_time_of_day)

# df['Weekday'] = df.index.weekday
# df['IsWeekend'] = df['Weekday'].apply(lambda x: 1 if x >= 5 else 0)
df.head()

,Temperature,Humidity,WindSpeed,IsHoliday,Year,Month,Day,Hour,Season,TimeOfDay
Date,,,,,,,,,,
2024-08-30 00:00:00,31.7,40,8.194444,0,2024,8,30,0,Summer,Night
2024-08-30 01:00:00,31.5,39,8.166667,0,2024,8,30,1,Summer,Night
2024-08-30 02:00:00,31.1,38,8.166667,0,2024,8,30,2,Summer,Night
2024-08-30 03:00:00,30.6,36,8.388889,0,2024,8,30,3,Summer,Night
2024-08-30 04:00:00,30.6,36,8.666667,0,2024,8,30,4,Summer,Night


In [14]:
df['Weekday'] = df.index.weekday  # Access weekday from index
# Add a column to indicate if it's a weekend
df['IsWeekend'] = df['Weekday'].apply(lambda x: 1 if x >= 4 else 0)

df = df.drop(columns = ['Weekday'])

In [15]:
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(df[['Season']])
df[ohe.get_feature_names_out(['Season'])] = ohe.transform(df[['Season']])

df['Season_Autumn']=0
df['Season_Spring']=0
df['Season_Summer']=0
df['Season_Winter']=0
ohe.fit(df[['TimeOfDay']])
df[ohe.get_feature_names_out(['TimeOfDay'])] = ohe.transform(df[['TimeOfDay']])

df = df.drop(columns = ['Season', 'TimeOfDay'])

df.head()

,Temperature,Humidity,WindSpeed,IsHoliday,Year,Month,Day,Hour,IsWeekend,Season_Autumn,Season_Summer,Season_Spring,Season_Winter,TimeOfDay_Afternoon,TimeOfDay_Evening,TimeOfDay_Morning,TimeOfDay_Night
Date,,,,,,,,,,,,,,,,,
2024-08-30 00:00:00,31.7,40,8.194444,0,2024,8,30,0,1,0,0,0,0,0.0,0.0,0.0,1.0
2024-08-30 01:00:00,31.5,39,8.166667,0,2024,8,30,1,1,0,0,0,0,0.0,0.0,0.0,1.0
2024-08-30 02:00:00,31.1,38,8.166667,0,2024,8,30,2,1,0,0,0,0,0.0,0.0,0.0,1.0
2024-08-30 03:00:00,30.6,36,8.388889,0,2024,8,30,3,1,0,0,0,0,0.0,0.0,0.0,1.0
2024-08-30 04:00:00,30.6,36,8.666667,0,2024,8,30,4,1,0,0,0,0,0.0,0.0,0.0,1.0


In [16]:
# Lag values (2, 4, 6, ..., 48 hours)
lags = [x for x in range(2,49,2)]

# Create lag features on the grouped (mean) data
# df_grouped = df.copy()

columns_to_lag = [
    'Temperature', 'Humidity', 'WindSpeed'
]

for col in columns_to_lag:
    for lag in lags:
        df[f'{col}_lag{lag}'] = df[col].shift(lag)

# Remove rows with NaN values due to lagging
df_lagged = df.dropna()
df_lagged.head()

,Temperature,Humidity,WindSpeed,IsHoliday,Year,Month,Day,Hour,IsWeekend,Season_Autumn,Season_Summer,Season_Spring,Season_Winter,TimeOfDay_Afternoon,TimeOfDay_Evening,TimeOfDay_Morning,TimeOfDay_Night,Temperature_lag2,Temperature_lag4,Temperature_lag6,Temperature_lag8,Temperature_lag10,Temperature_lag12,Temperature_lag14,Temperature_lag16,Temperature_lag18,Temperature_lag20,Temperature_lag22,Temperature_lag24,Temperature_lag26,Temperature_lag28,Temperature_lag30,Temperature_lag32,Temperature_lag34,Temperature_lag36,Temperature_lag38,Temperature_lag40,Temperature_lag42,Temperature_lag44,Temperature_lag46,Temperature_lag48,Humidity_lag2,Humidity_lag4,Humidity_lag6,Humidity_lag8,Humidity_lag10,Humidity_lag12,Humidity_lag14,Humidity_lag16,Humidity_lag18,Humidity_lag20,Humidity_lag22,Humidity_lag24,Humidity_lag26,Humidity_lag28,Humidity_lag30,Humidity_lag32,Humidity_lag34,Humidity_lag36,Humidity_lag38,Humidity_lag40,Humidity_lag42,Humidity_lag44,Humidity_lag46,Humidity_lag48,WindSpeed_lag2,WindSpeed_lag4,WindSpeed_lag6,WindSpeed_lag8,WindSpeed_lag10,WindSpeed_lag12,WindSpeed_lag14,WindSpeed_lag16,WindSpeed_lag18,WindSpeed_lag20,WindSpeed_lag22,WindSpeed_lag24,WindSpeed_lag26,WindSpeed_lag28,WindSpeed_lag30,WindSpeed_lag32,WindSpeed_lag34,WindSpeed_lag36,WindSpeed_lag38,WindSpeed_lag40,WindSpeed_lag42,WindSpeed_lag44,WindSpeed_lag46,WindSpeed_lag48
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-01 00:00:00,30.4,30,8.500000,0,2024,9,1,0,1,0,0,0,0,0.0,0.0,0.0,1.0,31.2,32.9,34.9,36.7,39.6,40.3,38.5,35.0,31.4,28.6,29.1,30.3,31.7,34.6,37.0,38.6,41.5,42.5,41.2,37.7,33.5,30.6,31.1,31.7,38.0,32.0,25.0,20.0,14.0,12.0,15.0,25.0,36.0,41.0,37.0,31.0,39.0,23.0,17.0,15.0,10.0,8.0,10.0,19.0,29.0,36.0,38.0,40.0,8.527778,9.000000,8.916667,8.166667,9.138889,9.472222,8.611111,9.638889,9.555556,8.833333,8.833333,8.527778,9.194444,8.916667,7.388889,7.055556,6.111111,8.833333,8.138889,8.250000,8.611111,8.666667,8.166667,8.194444
2024-09-01 01:00:00,30.8,31,8.805556,0,2024,9,1,1,1,0,0,0,0,0.0,0.0,0.0,1.0,30.9,31.9,33.7,35.8,38.5,39.9,39.7,36.9,32.9,29.7,28.5,29.8,30.7,33.2,35.7,37.7,40.5,41.7,42.2,39.7,35.5,31.7,30.6,31.5,32.0,37.0,29.0,22.0,16.0,13.0,13.0,20.0,33.0,40.0,41.0,34.0,41.0,31.0,21.0,16.0,12.0,11.0,8.0,14.0,24.0,34.0,36.0,39.0,8.527778,8.305556,9.250000,8.527778,8.805556,9.055556,8.888889,8.888889,9.916667,8.583333,8.888889,8.777778,8.527778,9.166667,8.305556,6.111111,6.277778,6.388889,8.000000,7.972222,8.444444,8.194444,8.388889,8.166667
2024-09-01 02:00:00,30.1,34,8.916667,0,2024,9,1,2,1,0,0,0,0,0.0,0.0,0.0,1.0,30.4,31.2,32.9,34.9,36.7,39.6,40.3,38.5,35.0,31.4,28.6,29.1,30.3,31.7,34.6,37.0,38.6,41.5,42.5,41.2,37.7,33.5,30.6,31.1,30.0,38.0,32.0,25.0,20.0,14.0,12.0,15.0,25.0,36.0,41.0,37.0,31.0,39.0,23.0,17.0,15.0,10.0,8.0,10.0,19.0,29.0,36.0,38.0,8.500000,8.527778,9.000000,8.916667,8.166667,9.138889,9.472222,8.611111,9.638889,9.555556,8.833333,8.833333,8.527778,9.194444,8.916667,7.388889,7.055556,6.111111,8.833333,8.138889,8.250000,8.611111,8.666667,8.166667
2024-09-01 03:00:00,29.5,39,8.805556,0,2024,9,1,3,1,0,0,0,0,0.0,0.0,0.0,1.0,30.8,30.9,31.9,33.7,35.8,38.5,39.9,39.7,36.9,32.9,29.7,28.5,29.8,30.7,33.2,35.7,37.7,40.5,41.7,42.2,39.7,35.5,31.7,30.6,31.0,32.0,37.0,29.0,22.0,16.0,13.0,13.0,20.0,33.0,40.0,41.0,34.0,41.0,31.0,21.0,16.0,12.0,11.0,8.0,14.0,24.0,34.0,36.0,8.805556,8.527778,8.305556,9.250000,8.527778,8.805556,9.055556,8.888889,8.888889,9.916667,8.583333,8.888889,8.777778,8.527778,9.166667,8.305556,6.111111,6.277778,6.388889,8.000000,7.972222,8.444444,8.194444,8.388889
2024-09-01 04:00:00,29.5,41,8.833333,0,2024,9,1,4,1,0,0,0,0,0.0,0.0,0.0,1.0,30.1,30.4,31.2,32.9,34.9,36.7,39.6,40.3,38.5,35.0,31.4,28.6,29.1,30.3,31.7,34.6,37.0,38.6,41.5,42.5,41.2,37.7,33.5,30.6,34.0,30.0,38.0,32.0,25.0,20.0,14.0,12.0,15.0,25.0,36.0,41.0,37.0,31.0,39.0,23.0,17.0,15.0,10.0,8.0,10.0,19.0,29.0,36.0,8.916667,8.500000,8.527778,9.000000,8.916667,8.166667,9.138889,9.472222,8.611111,9.638889,9.555556,8.83

In [17]:
df_lagged.columns

Index(['Temperature', 'Humidity', 'WindSpeed', 'IsHoliday', 'Year', 'Month',
       'Day', 'Hour', 'IsWeekend', 'Season_Autumn', 'Season_Summer',
       'Season_Spring', 'Season_Winter', 'TimeOfDay_Afternoon',
       'TimeOfDay_Evening', 'TimeOfDay_Morning', 'TimeOfDay_Night',
       'Temperature_lag2', 'Temperature_lag4', 'Temperature_lag6',
       'Temperature_lag8', 'Temperature_lag10', 'Temperature_lag12',
       'Temperature_lag14', 'Temperature_lag16', 'Temperature_lag18',
       'Temperature_lag20', 'Temperature_lag22', 'Temperature_lag24',
       'Temperature_lag26', 'Temperature_lag28', 'Temperature_lag30',
       'Temperature_lag32', 'Temperature_lag34', 'Temperature_lag36',
       'Temperature_lag38', 'Temperature_lag40', 'Temperature_lag42',
       'Temperature_lag44', 'Temperature_lag46', 'Temperature_lag48',
       'Humidity_lag2', 'Humidity_lag4', 'Humidity_lag6', 'Humidity_lag8',
       'Humidity_lag10', 'Humidity_lag12', 'Humidity_lag14', 'Humidity_lag16',
       'Humi

In [18]:
columns = ['Temperature', 'Humidity', 'WindSpeed', 'Year', 'Month', 'Day', 'Hour',
       'IsHoliday', 'IsWeekend', 'Season_Autumn', 'Season_Spring',
       'Season_Summer', 'Season_Winter', 'TimeOfDay_Afternoon',
       'TimeOfDay_Evening', 'TimeOfDay_Morning', 'TimeOfDay_Night',
       'Temperature_lag2', 'Temperature_lag4', 'Temperature_lag6',
       'Temperature_lag8', 'Temperature_lag10', 'Temperature_lag12',
       'Temperature_lag14', 'Temperature_lag16', 'Temperature_lag18',
       'Temperature_lag20', 'Temperature_lag22', 'Temperature_lag24',
       'Temperature_lag26', 'Temperature_lag28', 'Temperature_lag30',
       'Temperature_lag32', 'Temperature_lag34', 'Temperature_lag36',
       'Temperature_lag38', 'Temperature_lag40', 'Temperature_lag42',
       'Temperature_lag44', 'Temperature_lag46', 'Temperature_lag48',
       'Humidity_lag2', 'Humidity_lag4', 'Humidity_lag6', 'Humidity_lag8',
       'Humidity_lag10', 'Humidity_lag12', 'Humidity_lag14', 'Humidity_lag16',
       'Humidity_lag18', 'Humidity_lag20', 'Humidity_lag22', 'Humidity_lag24',
       'Humidity_lag26', 'Humidity_lag28', 'Humidity_lag30', 'Humidity_lag32',
       'Humidity_lag34', 'Humidity_lag36', 'Humidity_lag38', 'Humidity_lag40',
       'Humidity_lag42', 'Humidity_lag44', 'Humidity_lag46', 'Humidity_lag48',
       'WindSpeed_lag2', 'WindSpeed_lag4', 'WindSpeed_lag6', 'WindSpeed_lag8',
       'WindSpeed_lag10', 'WindSpeed_lag12', 'WindSpeed_lag14',
       'WindSpeed_lag16', 'WindSpeed_lag18', 'WindSpeed_lag20',
       'WindSpeed_lag22', 'WindSpeed_lag24', 'WindSpeed_lag26',
       'WindSpeed_lag28', 'WindSpeed_lag30', 'WindSpeed_lag32',
       'WindSpeed_lag34', 'WindSpeed_lag36', 'WindSpeed_lag38',
       'WindSpeed_lag40', 'WindSpeed_lag42', 'WindSpeed_lag44',
       'WindSpeed_lag46', 'WindSpeed_lag48']

df_lagged = df_lagged[columns]

In [19]:
df_lagged

,Temperature,Humidity,WindSpeed,Year,Month,Day,Hour,IsHoliday,IsWeekend,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,TimeOfDay_Afternoon,TimeOfDay_Evening,TimeOfDay_Morning,TimeOfDay_Night,Temperature_lag2,Temperature_lag4,Temperature_lag6,Temperature_lag8,Temperature_lag10,Temperature_lag12,Temperature_lag14,Temperature_lag16,Temperature_lag18,Temperature_lag20,Temperature_lag22,Temperature_lag24,Temperature_lag26,Temperature_lag28,Temperature_lag30,Temperature_lag32,Temperature_lag34,Temperature_lag36,Temperature_lag38,Temperature_lag40,Temperature_lag42,Temperature_lag44,Temperature_lag46,Temperature_lag48,Humidity_lag2,Humidity_lag4,Humidity_lag6,Humidity_lag8,Humidity_lag10,Humidity_lag12,Humidity_lag14,Humidity_lag16,Humidity_lag18,Humidity_lag20,Humidity_lag22,Humidity_lag24,Humidity_lag26,Humidity_lag28,Humidity_lag30,Humidity_lag32,Humidity_lag34,Humidity_lag36,Humidity_lag38,Humidity_lag40,Humidity_lag42,Humidity_lag44,Humidity_lag46,Humidity_lag48,WindSpeed_lag2,WindSpeed_lag4,WindSpeed_lag6,WindSpeed_lag8,WindSpeed_lag10,WindSpeed_lag12,WindSpeed_lag14,WindSpeed_lag16,WindSpeed_lag18,WindSpeed_lag20,WindSpeed_lag22,WindSpeed_lag24,WindSpeed_lag26,WindSpeed_lag28,WindSpeed_lag30,WindSpeed_lag32,WindSpeed_lag34,WindSpeed_lag36,WindSpeed_lag38,WindSpeed_lag40,WindSpeed_lag42,WindSpeed_lag44,WindSpeed_lag46,WindSpeed_lag48
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-01 00:00:00,30.4,30,8.500000,2024,9,1,0,0,1,0,0,0,0,0.0,0.0,0.0,1.0,31.2,32.9,34.9,36.7,39.6,40.3,38.5,35.0,31.4,28.6,29.1,30.3,31.7,34.6,37.0,38.6,41.5,42.5,41.2,37.7,33.5,30.6,31.1,31.7,38.0,32.0,25.0,20.0,14.0,12.0,15.0,25.0,36.0,41.0,37.0,31.0,39.0,23.0,17.0,15.0,10.0,8.0,10.0,19.0,29.0,36.0,38.0,40.0,8.527778,9.000000,8.916667,8.166667,9.138889,9.472222,8.611111,9.638889,9.555556,8.833333,8.833333,8.527778,9.194444,8.916667,7.388889,7.055556,6.111111,8.833333,8.138889,8.250000,8.611111,8.666667,8.166667,8.194444
2024-09-01 01:00:00,30.8,31,8.805556,2024,9,1,1,0,1,0,0,0,0,0.0,0.0,0.0,1.0,30.9,31.9,33.7,35.8,38.5,39.9,39.7,36.9,32.9,29.7,28.5,29.8,30.7,33.2,35.7,37.7,40.5,41.7,42.2,39.7,35.5,31.7,30.6,31.5,32.0,37.0,29.0,22.0,16.0,13.0,13.0,20.0,33.0,40.0,41.0,34.0,41.0,31.0,21.0,16.0,12.0,11.0,8.0,14.0,24.0,34.0,36.0,39.0,8.527778,8.305556,9.250000,8.527778,8.805556,9.055556,8.888889,8.888889,9.916667,8.583333,8.888889,8.777778,8.527778,9.166667,8.305556,6.111111,6.277778,6.388889,8.000000,7.972222,8.444444,8.194444,8.388889,8.166667
2024-09-01 02:00:00,30.1,34,8.916667,2024,9,1,2,0,1,0,0,0,0,0.0,0.0,0.0,1.0,30.4,31.2,32.9,34.9,36.7,39.6,40.3,38.5,35.0,31.4,28.6,29.1,30.3,31.7,34.6,37.0,38.6,41.5,42.5,41.2,37.7,33.5,30.6,31.1,30.0,38.0,32.0,25.0,20.0,14.0,12.0,15.0,25.0,36.0,41.0,37.0,31.0,39.0,23.0,17.0,15.0,10.0,8.0,10.0,19.0,29.0,36.0,38.0,8.500000,8.527778,9.000000,8.916667,8.166667,9.138889,9.472222,8.611111,9.638889,9.555556,8.833333,8.833333,8.527778,9.194444,8.916667,7.388889,7.055556,6.111111,8.833333,8.138889,8.250000,8.611111,8.666667,8.166667
2024-09-01 03:00:00,29.5,39,8.805556,2024,9,1,3,0,1,0,0,0,0,0.0,0.0,0.0,1.0,30.8,30.9,31.9,33.7,35.8,38.5,39.9,39.7,36.9,32.9,29.7,28.5,29.8,30.7,33.2,35.7,37.7,40.5,41.7,42.2,39.7,35.5,31.7,30.6,31.0,32.0,37.0,29.0,22.0,16.0,13.0,13.0,20.0,33.0,40.0,41.0,34.0,41.0,31.0,21.0,16.0,12.0,11.0,8.0,14.0,24.0,34.0,36.0,8.805556,8.527778,8.305556,9.250000,8.527778,8.805556,9.055556,8.888889,8.888889,9.916667,8.583333,8.888889,8.777778,8.527778,9.166667,8.305556,6.111111,6.277778,6.388889,8.000000,7.972222,8.444444,8.194444,8.388889
2024-09-01 04:00:00,29.5,41,8.833333,2024,9,1,4,0,1,0,0,0,0,0.0,0.0,0.0,1.0,30.1,30.4,31.2,32.9,34.9,36.7,39.6,40.3,38.5,35.0,31.4,28.6,29.1,30.3,31.7,34.6,37.0,38.6,41.5,42.5,41.2,37.7,33.5,30.6,34.0,30.0,38.0,32.0,25.0,20.0,14.0,12.0,15.0,25.0,36.0,41.0,37.0,31.0,39.0,23.0,17.0,15.0,10.0,8.0,10.0,19.0,29.0,36.0,8.916667,8.500000,8.527778,9.000000,8.916667,8.166667,9.138889,9.472222,8.611111,9.638889,9.555556,8.83

In [20]:
my_model = pickle.load(open("model3.pkl","rb"))

In [27]:
prediction = my_model.predict(df_lagged) * 6.359
prediction

array([1.16274482, 1.07297515, 1.07120731, 1.09348581, 1.09808634,
       1.07049027, 1.06887633, 1.09046415, 1.17787008, 1.25576868,
       1.3516547 , 1.39677424, 1.40224349, 1.40367441, 1.41031896,
       1.40906226, 1.40787879, 1.4091537 , 1.40921484, 1.58805927,
       1.63605778, 1.63064202, 1.61378367, 1.52062693, 1.16496379,
       1.08049413, 1.07572529, 1.10298836, 1.11321864, 1.08558521,
       1.0854694 , 1.10378677, 1.18634761, 1.2762326 , 1.37523858,
       1.41270191, 1.41733088, 1.41923867, 1.41964475, 1.41723934,
       1.41698204, 1.41701272, 1.41765512, 1.58929009, 1.63758452,
       1.63094595, 1.61339618, 1.52729707, 1.16599603, 1.07932738,
       1.07830636, 1.10188668, 1.11324005, 1.08676687, 1.08939825,
       1.10709701, 1.18423656, 1.27461593, 1.37391964, 1.41669065,
       1.41529044, 1.41344773, 1.41735438, 1.41866174, 1.41768757,
       1.41768757, 1.41774871, 1.58929009, 1.63758452, 1.63094595,
       1.61339618, 1.52729707, 1.16706227, 1.08119996, 1.07873

In [21]:
# Filter the DataFrame for 3 PM
df_filtered = df[(df['Hour'] == 15) & (df['TimeOfDay_Afternoon'] == 1)]

# Ensure there is at least one row
if not df_filtered.empty:
    row_to_predict = df_filtered.iloc[0]  # Get the first matching row

    # Send this row to the model for prediction
    prediction = my_model.predict([row_to_predict]) * 6.359

    # Display the prediction result
    print(f"Prediction for the row at 3 PM: {prediction}")
else:
    print("No row found for 3 PM.")


Prediction for the row at 3 PM: [1.2936194]


/Users/afnanalamri/.pyenv/versions/3.10.6/envs/Renewable_Energy/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
